In [ ]:
#Loading Packages
import pandas as pd 
import numpy as np                     # For mathematical calculations 
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings   # To ignore any warnings 
warnings.filterwarnings("ignore")
%matplotlib inline  
from mpl_toolkits.mplot3d import Axes3D


# Load Packages/Libraries


In [ ]:
# Dataset
df=pd.read_csv('../input/human-microbiome-project/project_catalog.csv')
df.head(20)

In [ ]:
df.info()

# EDA

In [ ]:
# Duplicates VALUE 

print(f'Duplicates in the dataset: {df.duplicated().sum()}')
print(f'Percentage of duplicates: {df.duplicated().sum()/len(df)*100}%')


In [ ]:
#Cardinality 
df.nunique()


In [ ]:
#Data Types 
df.dtypes


# Data Preprocessing and Visualization


In [ ]:
# Target Distribution
# Figure size 
plt.figure(figsize=(20,20))
# Pie plot
df['HMP Isolation Body Site'].value_counts().plot.pie(autopct='%1.1f%%', textprops={'fontsize':12}).set_title("Target distribution")


In [ ]:
# Target Distribution
# Figure size 
plt.figure(figsize=(20,20))
# Pie plot
df['Funding Source'].value_counts().plot.pie(autopct='%1.1f%%', textprops={'fontsize':12}).set_title("Target distribution")


In [ ]:

# Target Distribution
# Figure size 
plt.figure(figsize=(10,10))
# Pie plot
df['Project Status'].value_counts().plot.pie(autopct='%1.1f%%', textprops={'fontsize':12}).set_title("Target distribution")


In [ ]:
# Target Distribution
# Figure size 
plt.figure(figsize=(15,15))
# Pie plot
df['NCBI Submission Status'].value_counts().plot.pie(autopct='%1.1f%%', textprops={'fontsize':12}).set_title("Target distribution")


In [ ]:
df['Organism Name'].value_counts()


# Data interpretation

In [ ]:
df['Gene Count'].describe()


* Digging further it is evident that 1331 Bacteria have 0 gene count followed by 8 eukaryotes and 6 viruses.


In [ ]:
micro_gene_count=df[df['Gene Count']==0]
micro_gene_count['NCBI Superkingdom'].value_counts()


 * I was curious to check if the reason of 0 gene count is correlated with the project status. It appears that the reason behid 0 gene count for 1284bacteria, 4 eukaryotes, 1virus could be that the pojects are still in progress (when they released the data).



In [ ]:
micro_no_gene_progress= df[(df['Gene Count']==0) & (df['Project Status']=='In Progress')]
micro_no_gene_progress['NCBI Superkingdom'].value_counts()


* But for 47 bacteria, 5 viruses, 4 eukaryotes the project has been completed. Thus, the reason could be a reporting error.


In [ ]:
micro_no_gene_complete=df[(df['Gene Count']==0) & (df['Project Status']=='Complete')]
micro_no_gene_complete['NCBI Superkingdom'].value_counts()


In [ ]:
df[df['Gene Count']==8490]


* Next, I found that the research found 16 human body sites, with most diversity shown in the gastrointestinal tract.



In [ ]:
df['HMP Isolation Body Site'].nunique()


In [ ]:
df['HMP Isolation Body Site'].value_counts()


In [ ]:
plt.figure(figsize=(15,15))
sns.set_context('poster', font_scale=0.6)
df['HMP Isolation Body Site'].value_counts().plot(kind='bar')
plt.title('Distribution of microorganisms in various body sites')
plt.ylabel('Number of different microbes')
plt.xlabel('Human body sites')
plt.title('Diversity of microorganisms at different body sites')


In [ ]:
df['Genus']= df['Organism Name'].str.split(' ').str[0]
df['species']=df['Organism Name'].str.split(' ').str[1]
df[['Genus','species']].head()


In [ ]:
df['Genus'].nunique()


* There are 242 genera found, with Streptococcus being the most common genus in the human body.



In [ ]:
df['Genus'].value_counts().head(10)


* Before proceeding further, I checked for the unique values in 'NCBI Superkingdom' column. There are 3 observations labeled 'Error!!!' for this column.



In [ ]:
df.groupby('NCBI Superkingdom').count()


* As Streptococcus species ( which are bacteria) are the ones with 'Error!!!' in NCBI superkingdom, I have replaced this 'Error!!!' with 'Bacteria'.


In [ ]:
df[df['NCBI Superkingdom']=='Error!!!']


In [ ]:
df['NCBI Superkingdom'].replace('Error!!!', 'Bacteria', inplace=True)


* To proceed further, I wanted to fill the missing values in Domain and NCBI superkingdom columns. One can infer the result of one column if the other column's value is known.


In [ ]:
df[['Domain','NCBI Superkingdom']].isnull().sum()


* But we cannot fill the missing values in either of these columns, if both values of the 2 columns are missing. For that I checked how many such observations are present. Then I removed those observations.



In [ ]:
len(df.loc[df['Domain'].isnull()& df['NCBI Superkingdom'].isnull()])


In [ ]:
df=df.drop(df[(df['Domain'].isnull()) & (df['NCBI Superkingdom'].isnull())].index)
df.shape


* Bug - As many of the same rows have missing values for NCBI Superkingdom and Domain columns. The following groupby followed by transform steps were not working and giving an error message. To overcome that I first replaced missing value with the string 'NaN' in NCBI Superkingdom and then proceeded to the next step of filling missing value in Domain column ( groupby 'NCBI Superkingdom' and then transform it).



In [ ]:
df['NCBI Superkingdom'].fillna('NaN', inplace=True)
print(df.shape)
df['Domain'] =df.groupby('NCBI Superkingdom')['Domain'].transform(lambda x: x.fillna(x.mode().max()))
df['Domain'].isnull().sum()



* Next, I replaced the string 'NaN' using groupby and transform methods.


In [ ]:
df['NCBI Superkingdom']= df.groupby('Domain')['NCBI Superkingdom'].transform(lambda x: x.replace('NaN', x.mode().max()))
df.loc[df['NCBI Superkingdom']=='NaN']


* After getting rid of the missing values in 'NCBI Superkingdom' and 'Domain' columns, I was keen in checking where all the different types of microbes such as Bacteria, eukaryotes, viruses and archeae are located in the human body. From the given probing, it is clear that bacteria are located in all 16 studied human body sites followed by eukaryotes in 5 body sites, followed by viruses and archaea.



In [ ]:
df.groupby('NCBI Superkingdom')['HMP Isolation Body Site'].nunique().sort_values(ascending=False)


* As bacteria are more ubiquitous in the human body. The diversity is most vast in gastrointentestinal tract.



In [ ]:
bac=df.loc[df['Domain']=='BACTERIAL']
bac['HMP Isolation Body Site'].unique()


In [ ]:
bac['HMP Isolation Body Site'].value_counts(ascending=False).plot(kind='bar')
plt.ylabel('Number of different bacteria')
plt.xlabel('Human body sites')
plt.title('Diversity of bacteria at different body sites')


In [ ]:
euk=df.loc[df['Domain']=='EUKARYAL']
euk['HMP Isolation Body Site'].unique()


* More Eukaryotic diversity exist in the blood, followed by skin, airways, , wound, unknown.


In [ ]:
plt.figure(figsize=(10,10))
sns.set_context('poster', font_scale=0.6)
euk['HMP Isolation Body Site'].value_counts(ascending=False).plot(kind='bar')
plt.ylabel('Number of different eukaryotes')
plt.xlabel('Human body sites')
plt.title('Diversity of eukaryotes at different body sites')


* The study didn't find any precise location for viruses. Although, some previous studies have found virueses in blood, skin.



In [ ]:
vir=df.loc[df['Domain']=='VIRUS']
vir['HMP Isolation Body Site'].unique()


* Archaea is found primarily in the gastrointestinal tract.


In [ ]:
arc=df.loc[df['Domain']=='ARCHAEAL']
arc['HMP Isolation Body Site'].unique()


* Next question that comes to mind is which is the most ubiquitous organism found in this analysis. Staphylococcus is the most ubiquitous among all, with 11 habitats- urogenital_tract, skin, airways, unknown, gastrointestinal_tract, nose, blood, bone, eye, ear, other.



In [ ]:
plt.figure(figsize=(20,20))
sns.set_context('poster', font_scale=0.6)

z=df.groupby('Genus')['HMP Isolation Body Site'].nunique().sort_values(ascending=False)
y=pd.DataFrame(z)
w=y[y['HMP Isolation Body Site']>4]
print(w)
w.plot(kind='bar')
plt.ylabel('Number of different body sites')
plt.title('Number of habitats for different microorganisms')


In [ ]:
staph=df.loc[df['Genus']=='Staphylococcus']
staph['HMP Isolation Body Site'].unique()
df['NCBI Superkingdom'].value_counts()


* Just to mention that this study, includes 2882 Bacteria, 8 eukaryotes, 6 viruses and 2 archaea.



* Following is the list of names of viruses, eukaryotes and archaea.

In [ ]:
viruses= df[df['NCBI Superkingdom'] =='Viruses']
viruses['Organism Name']


In [ ]:
eukaryotes= df[df['NCBI Superkingdom']=='Eukaryota']
eukaryotes['Organism Name']


In [ ]:
archaea= df[df['NCBI Superkingdom']=='Archaea']
archaea['Organism Name']


# Conclusion
In this kernel we have performed EDA of the Human Microbiome dataset. Found- i) Gastrointestine shows most diversity of microbes, ii) Streptomyces sp. HGB0020 shows the maximum gene count in human, iii) Streptococcus is most common genus while Staphylococcus is most ubiquitous in humans.

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# project_catalog.csv has 2915 rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('../input/human-microbiome-project/project_catalog.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'project_catalog.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')


In [ ]:
df1.head(5)


* Distribution graphs (histogram/bar graph) of sampled columns:


In [ ]:
#plotPerColumnDistribution(df1, 12, 10)


# Correlation matrix


In [ ]:
plotCorrelationMatrix(df1, 8)


# Scatter and density plots


In [ ]:
plotScatterMatrix(df1, 20, 15)


# Pairplots

In [ ]:
#sns.set(rc={'figure.figsize':(30,30)})

sns.pairplot(df1, hue='HMP Isolation Body Site')


In [ ]:
sns.pairplot(df1, hue="HMP Isolation Body Site", diag_kind="hist")


In [ ]:
sns.pairplot(df1, kind="kde")


# heatmap

In [ ]:
df.corr()
plt.figure(figsize=(29,15))
sns.set_context('poster', font_scale=0.9)
sns.heatmap(df.corr(), cmap='coolwarm', annot=True)
#plt.title('R')
plt.show()


# Function for finding co-relation.


In [ ]:
# # Function for finding correlation.

# def corr_map(feature, size=((55, 55))):  
#   # Figure size
#   plt.figure(figsize=size)
#   sns.set_context('poster', font_scale= 1)

#   # Histogram
#   sns.histplot(data=df1, x=feature, hue='HMP Isolation Body Site', binwidth=1, kde=True)

#   # Aesthetics
#   plt.title(f'{feature} distribution')
#   plt.xlabel(f'{feature} Value')


In [ ]:
def corr_map(feature,size):  
  # Figure size
  plt.figure(figsize=size)
  sns.set_context('poster', font_scale= 1)

  # Histogram
  sns.histplot(data=df1, x=feature, hue='HMP Isolation Body Site', binwidth=1, kde=True)

  # Aesthetics
  plt.title(f'{feature} distribution')
  plt.xlabel(f'{feature} Value')


In [ ]:
corr_map('Funding Source',size=((30, 30)))


In [ ]:
corr_map('Sequencing Center',size=((35, 35)))


In [ ]:
corr_map('Project Status',size=((20, 20)))


In [ ]:

corr_map('HMP Isolation Body Site',size=((30, 30)))
